# 論文読解

以下の論文を読み問題に答えてください。CNNを使った物体検出（Object Detection）の代表的な研究です。


[8]Ren, S., He, K., Girshick, R., Sun, J.: Faster r-cnn: Towards real-time object detection with region proposal networks. In: Advances in neural information processing systems. (2015) 91–99


https://arxiv.org/pdf/1506.01497.pdf



## 問題
それぞれについてJupyter Notebookにマークダウン形式で記述してください。


(1) 物体検出の分野にはどういった手法が存在したか。


(2) Fasterとあるが、どういった仕組みで高速化したのか。


(3) One-Stageの手法とTwo-Stageの手法はどう違うのか。


(4) RPNとは何か。


(5) RoIプーリングとは何か。


(6) Anchorのサイズはどうするのが適切か。


(7) 何というデータセットを使い、先行研究に比べどういった指標値が得られているか。


(8) （アドバンス課題）Faster R-CNNよりも新しい物体検出の論文では、Faster R-CNNがどう引用されているか。


### 条件
- 答える際は論文のどの部分からそれが分かるかを書く。
- 必要に応じて先行研究（引用されている論文）も探しにいく。最低2つは他の論文を利用して回答すること。
- 論文の紹介記事を見ても良い。ただし、答えは論文内に根拠を探すこと。

##### Abstract
最新のオブジェクト検出ネットワークは、領域提案アルゴリズムに依存してオブジェクトの場所を仮定しています。 SPPnet [1]やFast R-CNN [2]などの進歩により、これらの検出ネットワークの実行時間が短縮され、領域提案の計算がボトルネックになっています。この作業では、検出ネットワークとフルイメージのたたみ込み機能を共有するリージョン提案ネットワーク（RPN）を導入し、ほぼ無料のリージョン提案を可能にします。 RPNは完全にたたみ込みネットワークで、各位置でオブジェクトの境界とオブジェクトネススコアを同時に予測します。 RPNはエンドツーエンドでトレーニングされ、高速R-CNNが検出に使用する高品質のリージョン提案を生成します。さらに、RPNとFast R-CNNを畳み込み機能を共有することで単一のネットワークに統合します。最近注目されているニューラルネットワークの用語に「注意」メカニズムを使用して、RPNコンポーネントは統合ネットワークにどこを見るかを指示します。非常に深いVGG-16モデル[3]の場合、PASCAL VOC 2007、2012、および最新のオブジェクト検出精度を達成しながら、GPUでの検出システムのフレームレートは5fps（すべてのステップを含む）です。画像あたり300の提案のみを含むMS COCOデータセット。 ILSVRCおよびCOCO 2015コンテストでは、Faster R-CNNおよびRPNは、いくつかのトラックで1位入賞エントリの基礎となります。コードは公開されています。

#### (1) 物体検出の分野にはどういった手法が存在したか。

Abstract 2行目より
- SPPnet

1INTRODUCTION 1-8行目より  
- 領域提案手法 (region proposal methods)  
- 領域ベースの畳み込みニューラルネットワーク（RCNN）  
-> 計算コストはかなり高くついた
- Fast R-CNN  
-> 提案間でたたみ込みを共有することにより、計算コストを大幅に削減し、高速化につながった

1INTRODUCTION 1段落目 10-12行目より  
現在、領域提案(region proposals)は、最先端の検出システムにおけるテスト時の計算上のボトルネックとなっている。

#### 解答  
SPPnetやFast R-CNNといった手法が検出にかかる時間を短縮するとともに、物体候補領域(物体が存在し得る画像領域の候補)がボトルネックであることを露呈した。

#### (2) Fasterとあるが、どういった仕組みで高速化したのか。

Abstract 4行目より  
RPNは完全にたたみ込みネットワークで、各位置でオブジェクトの境界とオブジェクトネススコアを同時に予測します。 RPNはエンドツーエンドでトレーニングされ、高速R-CNNが検出に使用する高品質のリージョン提案を生成します。さらに、RPNとFast R-CNNを畳み込み機能を共有することで単一のネットワークに統合します。

1INTRODUCTION 4段落目 8-10行目より  
テスト時に畳み込みを共有することで、提案を計算するための限界コストは小さくなります（たとえば、画像あたり10ミリ秒）。


#### 解答
Faster R-CNNはRPNとFast R-CNNを畳み込み機能を共有することで単一のネットワークに統合されたものである。  
Fast R-CNNで提案間でたたみ込みを共有することにより、計算コストを大幅に削減し、高速化につながったものの、領域提案(region proposals)は、最先端の検出システムにおけるテスト時の計算上のボトルネックとなっていた。  
そこでRPNという、エンドツーエンドで学習し、精度の良い物体候補領域(物体が存在し得る画像領域の候補)を抽出、提案する手法を組み込むことで高速化した。

#### (3) One-Stageの手法とTwo-Stageの手法はどう違うのか。

4 EXPERIMENTS p.10 One-Stage Detection vs. Two-Stage Proposal + Detection  
1段階検出と2段階提案+検出。 OverFeatペーパー[9]は、畳み込みフィーチャマップ上のスライディングウィンドウでリグレッサーと分類子を使用する検出方法を提案しています。 OverFeatは、1ステージのクラス固有の検出パイプラインであり、当社のクラスは、クラスに依存しない提案とクラス固有の検出で構成される2ステージのカスケードです。 OverFeatでは、領域ごとの機能は、スケールピラミッド上の1つのアスペクト比のスライディングウィンドウから取得されます。これらの機能は、オブジェクトの場所とカテゴリを同時に決定するために使用されます。 RPNでは、機能は正方形（3x3）のスライディングウィンドウからのものであり、異なるスケールとアスペクト比のアンカーに関連する提案を予測します。どちらの方法でもスライディングウィンドウを使用しますが、領域提案タスクはFaster RCNNの最初の段階にすぎません。ダウンストリームのFast R-CNN検出器は提案に精通してそれらを改良します。カスケードの第2段階では、地域ごとの機能が、地域の機能をより忠実にカバーする提案ボックスから適応的にプールされます[1]、[2]。これらの機能は、より正確な検出につながると信じています。



#### 解答  
One-Stageの手法はクラス特定の検出パイプラインのみからなる手法。
Two-Stageの手法はクラスに依存しない物体候補領域の提案とクラス特定の検出から構成される方法。

#### (4) RPNとは何か。

Abstract 4行目より  
RPNは完全にたたみ込みネットワークで、各位置でオブジェクトの境界とオブジェクトネススコアを同時に予測します。 RPNはエンドツーエンドでトレーニングされ、高速R-CNNが検出に使用する高品質のリージョン提案を生成します。さらに、RPNとFast R-CNNを畳み込み機能を共有することで単一のネットワークに統合します。


1INTRODUCTION 4段落目より  
このホワイトペーパーでは、アルゴリズムの変更、つまり深い畳み込みニューラルネットワークを使用した提案の計算が、検出ネットワークの計算を考えると、提案の計算がほぼ無料のエレガントで効果的なソリューションにつながることを示します。この目的を達成するために、最新のオブジェクト検出ネットワークと畳み込み層を共有する新しい領域提案ネットワーク（RPN）を紹介します[1]、[2]。テスト時に畳み込みを共有することで、提案を計算するための限界コストは小さくなります（たとえば、画像あたり10ミリ秒）。


5 CONCLUSION 1段落目 1-4行目より 
効率的で正確な領域提案生成のためのRPNを提示しました。下流検出ネットワークと畳み込みを共有することで、地域提案ステップはほぼコストフリーになります。

#### 解答  
RPNはエンドツーエンドで学習し、精度の良い物体候補領域(物体が存在し得る画像領域の候補)を抽出する、提案する手法

####  (5) RoIプーリングとは何か。

p.6 3.2 Sharing Features for RPN and Fast R-CNN (iii) Non-approximate joint training.より  
Fast R-CNNのRoIプーリングレイヤー[2]は、たたみ込み機能と予測される境界ボックスを入力として受け入れるため、理論的に有効な逆伝播ソルバーにも勾配w.r.tが必要です。ボックスの座標。上記の近似ジョイントトレーニングでは、これらの勾配は無視されます。非近似合同トレーニングソリューションでは、区別可能なRoIプーリングレイヤーが必要です。ボックスの座標。これは重要な問題であり、解決策は[15]で開発された「RoIワーピング」レイヤーによって提供できますが、このペーパーの範囲を超えています。

#### 解答  
Fast R-CNNのRoIプーリングレイヤーでは、たたみ込みを行った特徴量マップからRPNによって予測された境界ボックスの部分領域をプーリングして、固定サイズの特徴量マップとして抽出する。

#### (6) Anchorのサイズはどうするのが適切か。

p4. 3.1.1 Anchorsより  
アンカーは問題のスライディングウィンドウの中央にあり、スケールとアスペクト比に関連付けられています（図3、左）。デフォルトでは、3つのスケールと3つのアスペクト比を使用し、各スライド位置でk = 9個のアンカーを生成します。サイズW×H（通常〜2,400）の畳み込みフィーチャマップの場合、合計でW Hkアンカーがあります。

p4. Multi-Scale Anchors as Regression References 2段落目より  
比較として、アンカーベースの方法は、アンカーのピラミッドに基づいて構築されています。この方法では、複数のスケールとアスペクト比のアンカーボックスを参照して、境界ボックスを分類および回帰します。単一の縮尺の画像と機能マップのみに依存し、単一のサイズのフィルター（機能マップ上のスライドウィンドウ）を使用します。複数のスケールとサイズに対処するためのこのスキームの効果を実験で示します（表8）。

p6. 3.3 Implementation Details 2段落目より  
アンカーには、ボックス領域が1282、2562、および5122ピクセルの3つのスケールと、1：1、1：2、および2：1の3つのアスペクト比を使用します。これらのハイパーパラメータは、特定のデータセットに対して慎重に選択されていません。次のセクションで、それらの影響に関するアブレーション実験を提供します。

#### 解答  
デフォルトでは、3つのスケールと3つのアスペクト比を使用し、各スライド位置でk = 9個のアンカーを生成するが、これらは単一の縮尺の画像と特徴量マップのみに依存している。今回はボックス領域が1282、2562、および5122ピクセルの3つのスケールと、1：1、1：2、および2：1の3つのアスペクト比を使用している。

#### (7) 何というデータセットを使い、先行研究に比べどういった指標値が得られているか。

p.7 4.1 Experiments on PASCAL VOC より  
PASCAL VOC 2007検出ベンチマーク[11]に基づいて、私たちの方法を包括的に評価します。このデータセットは、20個のオブジェクトカテゴリにわたる約5kのtrainval画像と5kのテスト画像で構成されています。また、いくつかのモデルのPASCAL VOC 2012ベンチマークの結果も提供します。

これは主に検出平均の平均精度（mAP）を評価します。これは、オブジェクト提案のプロキシメトリックに焦点を当てるのではなく、オブジェクト検出の実際のメトリックだからです。

p.10 4.2 Experiments on MS COCO より  
Microsoft COCOオブジェクト検出データセット[12]について、さらに多くの結果を示します。このデータセットには、80のオブジェクトカテゴリが含まれます。トレーニングセットで80kの画像、検証セットで40kの画像、test-devセットで20kの画像を実験します。 IoU∈[0.5：0.05：0.95]（COCOの標準指標、単にmAP @ [。5、.95]と表記）とmAP@0.5（PASCAL VOCの指標）の平均mAPを評価します。

#### 解答  
PASCAL VOC 2007を主に使っており、いくつかのモデルにはcも試している。また、モデルの微調整をいくつかした上で、Microsoft COCOも使用している。評価に関しては検出平均の平均精度（mAP）で評価している。

<パラメータ>

- 物体検出
    - Fast R-CNN with ZF
    - Fast R-CNN and VGG-16 unshared
    - Fast R-CNN and VGG-16 sahred
    
- 領域提案の個数
    - 2000
    - 300

- 領域提案
    - SS
    - EB
    - RPN + ZF
    - RPN + VGG-16 unshared
    - RPN + VGG-16 shared
    
- 訓練データ 
    - PASCAL VOC 2007 
    - PASCAL VOC 2007 + PASCAL VOC 2012
    - Microsoft COCO + PASCAL VOC 2007 + PASCAL VOC 2012
    
- テストデータ
    - PASCAL VOC 2007 
    - PASCAL VOC 2012
    
    
概して領域提案にRPNを採用したモデルが精度が良くなっている。mAPが78.81と最も高い値を示したモデルは、

- 物体検出　Fast R-CNN and VGG-16 sahred
- 領域提案の個数　300
- 領域提案　RPN + VGG-16 shared
- 訓練データ　Microsoft COCO + PASCAL VOC 2007 + PASCAL VOC 2012
- テストデータ　PASCAL VOC 2007 


#### (8) （アドバンス課題）Faster R-CNNよりも新しい物体検出の論文では、Faster R-CNNがどう引用されているか。

